In [4]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, text, TIMESTAMP, func, DECIMAL, ForeignKey, CheckConstraint
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import os

In [5]:
# Parámetros de conexión
host = "localhost"
port = "5432"
dbname = "EcommerceDB"
user = "postgres"
password = "postgres123"

# Crear conexión
engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}", echo=True)
Session = sessionmaker(bind=engine)
session = Session()

In [6]:
Base = declarative_base()

class Usuario(Base):
    __tablename__ = 'usuarios'
    usuarioid = Column(Integer, primary_key=True, autoincrement=True)
    nombre = Column(String(100), nullable=False)
    apellido = Column(String(100), nullable=False)
    dni = Column(String(20), unique=True, nullable=False)
    email = Column(String(255), unique=True, nullable=False)
    contraseña = Column(String(255), nullable=False)
    fecharegistro = Column(TIMESTAMP, server_default=func.current_timestamp())

class Categoria(Base):
    __tablename__ = 'categorias'
    categoriaid = Column(Integer, primary_key=True, autoincrement=True)
    nombre = Column(String(100), nullable=False)
    descripcion = Column(String(255))

class Producto(Base):
    __tablename__ = 'productos'
    productoid = Column(Integer, primary_key=True, autoincrement=True)
    nombre = Column(String(255), nullable=False)
    descripcion = Column(String(255))
    precio = Column(DECIMAL(10, 2), nullable=False)
    stock = Column(Integer, nullable=False)
    categoriaid = Column(Integer, ForeignKey('categorias.categoriaid'), nullable=False)

class Orden(Base):
    __tablename__ = 'ordenes'
    ordenid = Column(Integer, primary_key=True, autoincrement=True)
    usuarioid = Column(Integer, ForeignKey('usuarios.usuarioid'), nullable=False)
    fechaorden = Column(TIMESTAMP, server_default=func.current_timestamp())
    total = Column(DECIMAL(10, 2), nullable=False)
    estado = Column(String(50), server_default='Pendiente')

class DetalleOrden(Base):
    __tablename__ = 'detalleordenes'
    detalleid = Column(Integer, primary_key=True, autoincrement=True)
    ordenid = Column(Integer, ForeignKey('ordenes.ordenid'), nullable=False)
    productoid = Column(Integer, ForeignKey('productos.productoid'), nullable=False)
    cantidad = Column(Integer, nullable=False)
    preciounitario = Column(DECIMAL(10, 2), nullable=False)

class DireccionEnvio(Base):
    __tablename__ = 'direccionesenvio'
    direccionid = Column(Integer, primary_key=True, autoincrement=True)
    usuarioid = Column(Integer, ForeignKey('usuarios.usuarioid'), nullable=False)
    calle = Column(String(255), nullable=False)
    ciudad = Column(String(100), nullable=False)
    departamento = Column(String(100))
    provincia = Column(String(100))
    distrito = Column(String(100))
    estado = Column(String(100))
    codigopostal = Column(String(20))
    pais = Column(String(100), nullable=False)

class Carrito(Base):
    __tablename__ = 'carrito'
    carritoid = Column(Integer, primary_key=True, autoincrement=True)
    usuarioid = Column(Integer, ForeignKey('usuarios.usuarioid'), nullable=False)
    productoid = Column(Integer, ForeignKey('productos.productoid'), nullable=False)
    cantidad = Column(Integer, nullable=False)
    fechaagregado = Column(TIMESTAMP, server_default=func.current_timestamp())

class MetodoPago(Base):
    __tablename__ = 'metodospago'
    metodopagoid = Column(Integer, primary_key=True, autoincrement=True)
    nombre = Column(String(100), nullable=False)
    descripcion = Column(String(255))

class OrdenMetodoPago(Base):
    __tablename__ = 'ordenesmetodospago'
    ordenmetodoid = Column(Integer, primary_key=True, autoincrement=True)
    ordenid = Column(Integer, ForeignKey('ordenes.ordenid'), nullable=False)
    metodopagoid = Column(Integer, ForeignKey('metodospago.metodopagoid'), nullable=False)
    montopagado = Column(DECIMAL(10, 2), nullable=False)

class ReseñaProducto(Base):
    __tablename__ = 'reseñasproductos'
    reseñaid = Column(Integer, primary_key=True, autoincrement=True)
    usuarioid = Column(Integer, ForeignKey('usuarios.usuarioid'), nullable=False)
    productoid = Column(Integer, ForeignKey('productos.productoid'), nullable=False)
    calificacion = Column(Integer, CheckConstraint('calificacion >= 1 AND calificacion <= 5'))
    comentario = Column(String(255))
    fecha = Column(TIMESTAMP, server_default=func.current_timestamp())

class HistorialPago(Base):
    __tablename__ = 'historialpagos'
    pagoid = Column(Integer, primary_key=True, autoincrement=True)
    ordenid = Column(Integer, ForeignKey('ordenes.ordenid'), nullable=False)
    metodopagoid = Column(Integer, ForeignKey('metodospago.metodopagoid'), nullable=False)
    monto = Column(DECIMAL(10, 2), nullable=False)
    fechapago = Column(TIMESTAMP, server_default=func.current_timestamp())
    estadopago = Column(String(50), server_default='Procesando')

Base.metadata.drop_all(engine)
Base.metadata.create_all(engine)


2025-07-24 01:46:04,057 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-07-24 01:46:04,058 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-07-24 01:46:04,060 INFO sqlalchemy.engine.Engine select current_schema()
2025-07-24 01:46:04,061 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-07-24 01:46:04,062 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-07-24 01:46:04,063 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-07-24 01:46:04,064 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-24 01:46:04,069 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

C:\Users\gabriel.b.alvarez\AppData\Local\Temp\ipykernel_17256\2515579819.py:1: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


2025-07-24 01:46:04,187 INFO sqlalchemy.engine.Engine [no key 0.00141s] {}
2025-07-24 01:46:04,188 INFO sqlalchemy.engine.Engine 
CREATE TABLE historialpagos (
	pagoid SERIAL NOT NULL, 
	ordenid INTEGER NOT NULL, 
	metodopagoid INTEGER NOT NULL, 
	monto DECIMAL(10, 2) NOT NULL, 
	fechapago TIMESTAMP WITHOUT TIME ZONE DEFAULT CURRENT_TIMESTAMP, 
	estadopago VARCHAR(50) DEFAULT 'Procesando', 
	PRIMARY KEY (pagoid), 
	FOREIGN KEY(ordenid) REFERENCES ordenes (ordenid), 
	FOREIGN KEY(metodopagoid) REFERENCES metodospago (metodopagoid)
)


2025-07-24 01:46:04,188 INFO sqlalchemy.engine.Engine [no key 0.00072s] {}
2025-07-24 01:46:04,195 INFO sqlalchemy.engine.Engine COMMIT


In [7]:
ruta_base = r"DB Proyecto\csv\CSV-20250718T195209Z-1-001\CSV"

map_csv_table = {
    "2.Usuarios.csv": "usuarios",
    "3.Categorias.csv": "categorias",
    "4.Productos.csv": "productos",
    "5.ordenes.csv": "ordenes",
    "6.detalle_ordenes.csv": "detalleordenes",
    "7.direcciones_envio.csv": "direccionesenvio",
    "8.carrito.csv": "carrito",
    "9.metodos_pago.csv": "metodospago",
    "10.ordenes_metodospago.csv": "ordenesmetodospago",
    "11.resenas_productos.csv": "reseñasproductos",
    "12.historial_pagos.csv": "historialpagos"
}

for file_name, table_name in map_csv_table.items():
    file_path = os.path.join(ruta_base, file_name)
    df = pd.read_csv(file_path)
    df.columns = df.columns.str.lower()
    df.to_sql(table_name, con=engine, if_exists='append', index=False)


2025-07-24 01:46:06,061 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-24 01:46:06,064 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2025-07-24 01:46:06,064 INFO sqlalchemy.engine.Engine [cached since 1.997s ago] {'table_name': 'usuarios', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2025-07-24 01:46:06,077 INFO sqlalchemy.engine.Engine INSERT INTO usuarios (nombre, apellido, dni, email, "contraseña") VALUES (%(nombre__0)s, %(apellido__0)s, %(dni__0)s, %(email__0)s, %(contraseña__0)s), (%(nombre__1)s, %(apell